In [2]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain_huggingface import HuggingFaceEndpoint


In [15]:
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
KEY = os.getenv["HUGGINGFACEHUB_API_TOKEN"]

TypeError: 'function' object is not subscriptable

In [13]:
model_name = "google/byt5-small"
model = HuggingFaceEndpoint(
    repo_id=model_name,
    huggingfacehub_api_token=KEY,
    temperature= 0
)